# Constained Binary Optimization with Commute Hamiltonian-based QAOA

**Author:** Debin Xiang & Qifan Jiang

**Date:** 02/02/2025

Based on paper "[Choco-Q: Commute Hamiltonian-based QAOA for Constrained Binary Optimization][1]" (HPCA 2025)

[1]: https://ieeexplore.ieee.org/document/TBD

Constrained binary optimization aims to find an optimal assignment to minimize or maximize the objective meanwhile satisfying the constraints, which is a representative NP problem in various domains, including transportation, scheduling, and economy. Quantum approximate optimization algorithms (QAOA) provide a promising methodology for solving this problem by exploiting the parallelism of quantum entanglement. However, existing QAOA approaches based on penalty-term or Hamiltonian simulation fail to thoroughly encode the constraints, leading to extremely low success rate and long searching latency.

In this notebook, we introduce Choco-Q, a formal and universal framework for constrained binary optimization problems, which comprehensively covers all constraints and exhibits high deployability for current quantum devices.

<div style="text-align:center;">
    <img src="../picture/6_1_commute_constrained.jpg"  width="45%" height="45%">
</div>

The main innovation of Choco-Q is to embed the commute Hamiltonian as the driver Hamiltonian, resulting in a much more general encoding formulation that can deal with arbitrary linear constraints. In quantum mechanics, Heisenberg’s picture states that if the Hamiltonian commutes with an operator, the expectation of the operator remains invariant during the evolution. With this understanding, we can deduce that if a Hamiltonian commutes with the constraints operator, the states will be restricted to a subspace where each state adheres to the constraints, as shown in the figure above.

**Execute this notebook with the virtual environment for Choco-Q to avoid package conflicts.**

In [1]:
import os
os.chdir("..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

from janusq.application.chocoq.chocoq.model import LinearConstrainedBinaryOptimization as LcboModel
from janusq.application.chocoq.chocoq.solvers.optimizers import CobylaOptimizer, AdamOptimizer
from janusq.application.chocoq.chocoq.solvers.qiskit import (
    PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver, 
    AerGpuProvider, AerProvider, FakeBrisbaneProvider, FakeKyivProvider, FakeTorinoProvider, DdsimProvider,
)

## Configure the problem

In [2]:
# configure the constrained binary optimization with LcboModel
m = LcboModel()
x = m.addVars(5, name="x") # add five binary variables with prefix name x
m.setObjective((x[0] + x[1])* x[3] + x[2], "max") # set the maximum objective with (x[0]+x[1])x[3] + x[2]
m.addConstr(x[0] + x[1] - x[2] == 0) # add linear constraints
m.addConstr(x[2] + x[3] - x[4] == 1)

print(m)
print(f"Linear Constraints Matrix:\n{m.lin_constr_mtx}\n")

optimize = m.optimize()
print(f"Classical optimized cost and assignment: {optimize}\n")

Model:
- variables: x_0 (type: binary)   x_1 (type: binary)   x_2 (type: binary)   x_3 (type: binary)   x_4 (type: binary)
- obj: max 1 * x_0 * x_3 + 1 * x_1 * x_3 + 1 * x_2
- s.t.:
1 * x_0 + 1 * x_1 + -1 * x_2 == 0
1 * x_2 + 1 * x_3 + -1 * x_4 == 1
- penalty_lambda: 32767

Linear Constraints Matrix:
[[ 1  1 -1  0  0  0]
 [ 0  0  1  1 -1  1]]

Restricted license - for non-production use only - expires 2026-11-23
Classical optimized cost and assignment: (2.0, {'x_0': 0.0, 'x_1': 1.0, 'x_2': 1.0, 'x_3': 1.0, 'x_4': 1.0})



## Initialize Choco-Q solver

In [3]:
# set the optimizer and provider for Choco-Q
opt = CobylaOptimizer(max_iter=200) # set the optimizer COBYLA, the max iteration is 200.
# gpu = AerGpuProvider()
aer = DdsimProvider()
solver = ChocoSolver(
    prb_model=m,  # the problem model
    optimizer=opt,  # the optimizer for parameters updating.
    provider=aer,  # quantum circuit simulator.
    num_layers=1,
)

## analyze the circuit
circuit_depth, circuit_width, culled_depth, num_one_qubit_gates = solver.circuit_analyze(['depth', 'width', 'culled_depth', 'num_one_qubit_gates'])
print(f"Circuit Depth: {circuit_depth}")
print(f"Circuit Width: {circuit_width}")
print(f"Circuit Culled Depth: {culled_depth}")
print(f"Circuit Num One Qubit Gates: {num_one_qubit_gates}")

Circuit Depth: 127
Circuit Width: 12
Circuit Culled Depth: 62
Circuit Num One Qubit Gates: None


## Using Choco-Q API to solve the problem

In [4]:
# solve by Choco-Q
state_list, prob_list, iter_count = solver.solve()
result_dict = {prob: state for state, prob in zip(state_list, prob_list)}
print(f"Quantum optimized assignment:")
for prob, state in result_dict.items():
    print(f"Solution Probability: {prob}, Assignment: {state}")

Quantum optimized assignment:
Solution Probability: 0.046875, Assignment: [0, 0, 0, 1, 0]
Solution Probability: 0.7861328125, Assignment: [1, 0, 1, 0, 0]
Solution Probability: 0.1669921875, Assignment: [1, 0, 1, 1, 1]


In [5]:
# evaluate the solution
best_solution_prob, in_constraints_prob, ARG, iter_count = solver.evaluation()
print(f"Best Solution Probability: {best_solution_prob}")
print(f"In Constraints Probability: {in_constraints_prob}")
print(f"ARG: {ARG}")
print(f"Iteration count: {iter_count}")

Best Solution Probability: 16.69921875
In Constraints Probability: 100.0
ARG: 0.439941404050293
Iteration count: 29
